In [2]:
import pandas as pd

copi_complex_df = pd.read_csv(r"../data/COPI_complex/heklopit_pl3017_frd1ppi_sc151_fdr1rp_COPI.csv")
copi_complex_df.head()

,protein_a,protein_b,gene_a,gene_b,pepA,pepB,Score,Software
0,P53618,P35606,COPB1,COPB2,AAAQCYIDLII[K]ESDNNVK,AMGDAEI[K]DGER,0.999972,Plink3
1,P53618,P35606,COPB1,COPB2,AAAQCYIDLII[K]ESDNNVK,AMGDAEI[K]DGERLPLAVK,1.000000,Plink3
2,P53618,P53618,COPB1,COPB1,AAAQCYIDLII[K]ESDNNVK,EVI[K]TNNVSEHEDTDK,0.999989,Plink3
3,P53618,P48444,COPB1,ARCN1,AAAQCYIDLII[K]ESDNNVK,G[K]EVDNFVDK,0.997517,Plink3
4,P53618,P35606,COPB1,COPB2,AAAQCYIDLII[K]ESDNNVK,VAHFLE[K]QGFK,0.999382,Plink3


In [ ]:
# 迭代每一行，如果有；分开的，先删除，降低复杂度


In [6]:
from Bio import SeqIO

copi_fasta_path = r"../data/COPI_complex/COPI.fasta"

copi_seqs_dict = {}
with open(copi_fasta_path) as handle:
    for record in SeqIO.parse(handle, "fasta"):
        protein_id = str(record.id).split("|")[1]
        seq = str(record.seq)
        copi_seqs_dict[protein_id] = seq

copi_seqs_dict


{'O14579': 'MAPPAPGPASGGSGEVDELFDVKNAFYIGSYQQCINEAQRVKLSSPERDVERDVFLYRAYLAQRKFGVVLDEIKPSSAPELQAVRMFADYLAHESRRDSIVAELDREMSRSVDVTNTTFLLMAASIYLHDQNPDAALRALHQGDSLECTAMTVQILLKLDRLDLARKELKRMQDLDEDATLTQLATAWVSLATGGEKLQDAYYIFQEMADKCSPTLLLLNGQAACHMAQGRWEAAEGLLQEALDKDSGYPETLVNLIVLSQHLGKPPEVTNRYLSQLKDAHRSHPFIKEYQAKENDFDRLVLQYAPSA',
 'P35606': 'MPLRLDIKRKLTARSDRVKSVDLHPTEPWMLASLYNGSVCVWNHETQTLVKTFEVCDLPVRAAKFVARKNWVVTGADDMQIRVFNYNTLERVHMFEAHSDYIRCIAVHPTQPFILTSSDDMLIKLWDWDKKWSCSQVFEGHTHYVMQIVINPKDNNQFASASLDRTIKVWQLGSSSPNFTLEGHEKGVNCIDYYSGGDKPYLISGADDRLVKIWDYQNKTCVQTLEGHAQNVSCASFHPELPIIITGSEDGTVRIWHSSTYRLESTLNYGMERVWCVASLRGSNNVALGYDEGSIIVKLGREEPAMSMDANGKIIWAKHSEVQQANLKAMGDAEIKDGERLPLAVKDMGSCEIYPQTIQHNPNGRFVVVCGDGEYIIYTAMALRNKSFGSAQEFAWAHDSSEYAIRESNSIVKIFKNFKEKKSFKPDFGAESIYGGFLLGVRSVNGLAFYDWDNTELIRRIEIQPKHIFWSDSGELVCIATEESFFILKYLSEKVLAAQETHEGVTEDGIEDAFEVLGEIQEIVKTGLWVGDCFIYTSSVNRLNYYVGGEIVTIAHLDRTMYLLGYIPKDNRLYLGDKELNIISYSLLVSVLEYQTAVMRRDFSMADKVLPTIPKEQRTRVAHFLEKQGFKQQALTVSTDPEHRFELALQLGELKIAYQLAVEAE

In [7]:
def find_bracket_k_position(peptide, protein):
    # 去除方括号，只保留氨基酸序列
    clean_pep = peptide.replace("[K]", "K")

    # 找到短肽在蛋白中的起始位置
    start = protein.find(clean_pep)
    if start == -1:
        return None

    # 找到 [K] 在短肽中的位置（基于干净序列）
    k_index_in_pep = clean_pep.index("K")  # 0-based

    # 蛋白中的位置（0-based）
    k_pos_0 = start + k_index_in_pep

    # 返回 1-based 的位置
    return k_pos_0 + 1


find_bracket_k_position("AAAQCYIDLII[K]ESDNNVK",copi_seqs_dict["P53618"])

291

In [23]:
json_files = {}
for index,row in copi_complex_df.iterrows():
    try:
        if f"{row[0]}_{row[1]}" in json_files.keys():
            crosslink_residue_pair = [
                    [[str(row[2]), find_bracket_k_position(row["pepA"],copi_seqs_dict[row["protein_a"]])],
                    [str(row[3]), find_bracket_k_position(row["pepB"],copi_seqs_dict[row["protein_b"]])]]
                    ]
            json_files[f"{row[0]}_{row[1]}"]["crosslinks"][0]["residue_pairs"].append(crosslink_residue_pair)
            # break
        elif f"{row[1]}_{row[0]}" in json_files.keys():
            crosslink_residue_pair = [
                    [
                    [str(row[3]), find_bracket_k_position(row["pepB"],copi_seqs_dict[row["protein_b"]])]],
                    [str(row[2]), find_bracket_k_position(row["pepA"],copi_seqs_dict[row["protein_a"]])]
                    ]
            json_files[f"{row[1]}_{row[0]}"]["crosslinks"][0]["residue_pairs"].append(crosslink_residue_pair)
        else:
            json_files[f"{row[0]}_{row[1]}"] = {}
            json_files[f"{row[0]}_{row[1]}"]["name"] = f"{row[0]}_{row[1]}"
            json_files[f"{row[0]}_{row[1]}"]["modelSeeds"] = [1]

            protein_a = {"id":str(row[2]),
                        "sequence":str(copi_seqs_dict[row[0]])}
            protein_b = {"id":str(row[3]),
                        "sequence":str(copi_seqs_dict[row[1]])}
            json_files[f"{row[0]}_{row[1]}"]["sequences"] = [protein_a,protein_b]
            json_files[f"{row[0]}_{row[1]}"]["dialect"] = "alphafold3"
            json_files[f"{row[0]}_{row[1]}"]["version"] = 1

            json_files[f"{row[0]}_{row[1]}"]["crosslinks"] = [{
                "name": "azide-A-DSBSO",
                "residue_pairs":[
                    [[str(row[2]), find_bracket_k_position(row["pepA"],copi_seqs_dict[row["protein_a"]])],
                    [str(row[3]), find_bracket_k_position(row["pepB"],copi_seqs_dict[row["protein_b"]])]]
                    ]
            }]

    except:
        pass
    

json_files

C:\Users\zhang.zehong\AppData\Local\Temp\ipykernel_31484\3596784885.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if f"{row[0]}_{row[1]}" in json_files.keys():
C:\Users\zhang.zehong\AppData\Local\Temp\ipykernel_31484\3596784885.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif f"{row[1]}_{row[0]}" in json_files.keys():
C:\Users\zhang.zehong\AppData\Local\Temp\ipykernel_31484\3596784885.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, us

{'P53618_P35606': {'name': 'P53618_P35606',
  'modelSeeds': [1],
  'sequences': [{'id': 'COPB1',
    'sequence': 'MTAAENVCYTLINVPMDSEPPSEISLKNDLEKGDVKSKTEALKKVIIMILNGEKLPGLLMTIIRFVLPLQDHTIKKLLLVFWEIVPKTTPDGRLLHEMILVCDAYRKDLQHPNEFIRGSTLRFLCKLKEAELLEPLMPAIRACLEHRHSYVRRNAVLAIYTIYRNFEHLIPDAPELIHDFLVNEKDASCKRNAFMMLIHADQDRALDYLSTCIDQVQTFGDILQLVIVELIYKVCHANPSERARFIRCIYNLLQSSSPAVKYEAAGTLVTLSSAPTAIKAAAQCYIDLIIKESDNNVKLIVLDRLIELKEHPAHERVLQDLVMDILRVLSTPDLEVRKKTLQLALDLVSSRNVEELVIVLKKEVIKTNNVSEHEDTDKYRQLLVRTLHSCSVRFPDMAANVIPVLMEFLSDNNEAAAADVLEFVREAIQRFDNLRMLIVEKMLEVFHAIKSVKIYRGALWILGEYCSTKEDIQSVMTEIRRSLGEIPIVESEIKKEAGELKPEEEITVGPVQKLVTEMGTYATQSALSSSRPTKKEEDRPPLRGFLLDGDFFVAASLATTLTKIALRYVALVQEKKKQNSFVAEAMLLMATILHLGKSSLPKKPITDDDVDRISLCLKVLSECSPLMNDIFNKECRQSLSHMLSAKLEEEKLSQKKESEKRNVTVQPDDPISFMQLTAKNEMNCKEDQFQLSLLAAMGNTQRKEAADPLASKLNKVTQLTGFSDPVYAEAYVHVNQYDIVLDVLVVNQTSDTLQNCTLELATLGDLKLVEKPSPLTLAPHDFANIKANVKVASTENGIIFGNIVYDVSGAASDRNCVVLSDIHIDIMDYIQPATCTDAEFRQMWAEFEWENKVTVNTNMVDLNDYLQHILKSTNMKCLTPEKALSG

In [32]:
json_files = {}

def build_residue_pair(row, A_first=True):
    """生成 residue_pairs，自动处理 A/B 方向"""
    if A_first:
        return [
            ["A", find_bracket_k_position(row["pepA"], copi_seqs_dict[row["protein_a"]])],
            ["B", find_bracket_k_position(row["pepB"], copi_seqs_dict[row["protein_b"]])]
        ]
    else:
        return [
            ["A", find_bracket_k_position(row["pepB"], copi_seqs_dict[row["protein_b"]])],
            ["B", find_bracket_k_position(row["pepA"], copi_seqs_dict[row["protein_a"]])]
        ]

for _, row in copi_complex_df.iterrows():
    try:
        key_direct = f"{row[2]}_{row[3]}"
        key_reverse = f"{row[3]}_{row[2]}"
        if key_direct != key_reverse:

            # 判断 key 方向
            if key_direct in json_files:
                key = key_direct
                A_first = True
            elif key_reverse in json_files:
                key = key_reverse
                A_first = False
            else:
                # 创建新 entry
                key = key_direct
                json_files[key] = {
                    "name": key,
                    "modelSeeds": [1],
                    "sequences": [
                        {"protein":{"id": "A", "sequence": str(copi_seqs_dict[row[0]])}},
                        {"protein":{"id": "B", "sequence": str(copi_seqs_dict[row[1]])}}
                    ],
                    "dialect": "alphafold3",
                    "version": 1,
                    "crosslinks": [{
                        "name": "azide-A-DSBSO",
                        "residue_pairs": []
                    }]
                }
                A_first = True

            # 追加 residue pair
            residue_pair = build_residue_pair(row, A_first)
            json_files[key]["crosslinks"][0]["residue_pairs"].append(residue_pair)

    except Exception as e:
        print("Error:", e)  # 建议保留错误信息，不要 pass

json_files


Error: 'Q9UBF2; Q9Y678'
Error: 'Q9UBF2; Q9Y678'
Error: 'Q9UBF2; Q9Y678'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'P00966; P48444'
Error: 'Q9UBF2; Q9Y678'
Error: 'Q9UBF2; Q9Y678'
Error: 'Q9UBF2; Q9Y678'
Error: 'Q9UBF2; Q9Y678'
Error: 'Q9UBF2; Q9Y678'
Error: 'Q9UBF2; Q9Y678'
Error: 'Q8N6H7; Q9NP61'
Error: 'Q8IYB5; Q8N6H7'
Error: 'Q8IYB5; Q8N6H7'
Error: 'Q8IYB5; Q8N6H7'
Error: 'Q8IYB5; Q8N6H7'
Error: 'Q8IYB5; Q8N6H7'
Error: 'Q8IYB5; Q8N6H7'
Error: 'Q8IYB5; Q8N6H7'
Error: 'O75717; P53618'
Error: 'O75717; 

C:\Users\zhang.zehong\AppData\Local\Temp\ipykernel_31484\3246703713.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  key_direct = f"{row[2]}_{row[3]}"
C:\Users\zhang.zehong\AppData\Local\Temp\ipykernel_31484\3246703713.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  key_reverse = f"{row[3]}_{row[2]}"
C:\Users\zhang.zehong\AppData\Local\Temp\ipykernel_31484\3246703713.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  {"pr

{'COPB1_COPB2': {'name': 'COPB1_COPB2',
  'modelSeeds': [1],
  'sequences': [{'protein': {'id': 'A',
     'sequence': 'MTAAENVCYTLINVPMDSEPPSEISLKNDLEKGDVKSKTEALKKVIIMILNGEKLPGLLMTIIRFVLPLQDHTIKKLLLVFWEIVPKTTPDGRLLHEMILVCDAYRKDLQHPNEFIRGSTLRFLCKLKEAELLEPLMPAIRACLEHRHSYVRRNAVLAIYTIYRNFEHLIPDAPELIHDFLVNEKDASCKRNAFMMLIHADQDRALDYLSTCIDQVQTFGDILQLVIVELIYKVCHANPSERARFIRCIYNLLQSSSPAVKYEAAGTLVTLSSAPTAIKAAAQCYIDLIIKESDNNVKLIVLDRLIELKEHPAHERVLQDLVMDILRVLSTPDLEVRKKTLQLALDLVSSRNVEELVIVLKKEVIKTNNVSEHEDTDKYRQLLVRTLHSCSVRFPDMAANVIPVLMEFLSDNNEAAAADVLEFVREAIQRFDNLRMLIVEKMLEVFHAIKSVKIYRGALWILGEYCSTKEDIQSVMTEIRRSLGEIPIVESEIKKEAGELKPEEEITVGPVQKLVTEMGTYATQSALSSSRPTKKEEDRPPLRGFLLDGDFFVAASLATTLTKIALRYVALVQEKKKQNSFVAEAMLLMATILHLGKSSLPKKPITDDDVDRISLCLKVLSECSPLMNDIFNKECRQSLSHMLSAKLEEEKLSQKKESEKRNVTVQPDDPISFMQLTAKNEMNCKEDQFQLSLLAAMGNTQRKEAADPLASKLNKVTQLTGFSDPVYAEAYVHVNQYDIVLDVLVVNQTSDTLQNCTLELATLGDLKLVEKPSPLTLAPHDFANIKANVKVASTENGIIFGNIVYDVSGAASDRNCVVLSDIHIDIMDYIQPATCTDAEFRQMWAEFEWENKVTVNTNMVDLNDYLQHILKSTNMKCLTPE

In [33]:
import json
for key in json_files.keys():
    file_name = f"../data/COPI_complex/jsons/{key}.json"
    data = json_files[key]
    with open(file_name, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

1. 两两进行预测，取有最多复合物相差最小的单体作为模板。再用该模板将所有相关复合物预测一遍
2. 最终进行全面拼装组合
3. 对力场进行微调，更加符合真实结构，解决冲突